## Sweep bot

i'm fucking crashing out
why is nothing fucking working

In [ ]:
%load_ext autoreload
%autoreload 2
# ^ make it so we can change inner files


In [ ]:

import numpy as np
from pydrake.all import (
    ContactModel,
    Demultiplexer,
    DiagramBuilder,
    InverseDynamicsController,
    MeshcatVisualizer,
    MultibodyPlant,
    Rgba,
    RigidTransform,
    RollPitchYaw,
    RotationMatrix,
    SceneGraph,
    Simulator,
    Sphere,
    StartMeshcat,
    VectorLogSink,
    VisualizationConfig,
)

from manipulation.meshcat_utils import (
    AddMeshcatTriad,
)
from manipulation.station import (
    AddPointClouds,
    MakeHardwareStation,
    Parser,
)
from scripts.load_scenario import load_scenario
from scripts.grasp_broom import plan_path, grasp_path, build_temp_plant
from scripts.meta_controller import MetaController

In [ ]:
# Start meshcat for visualization
meshcat = StartMeshcat()
print("Click the link above to open Meshcat in your browser!")

In [ ]:
use_hydroelastic = True # this takes forever

def make_diagram():

    scenario = load_scenario()

    # since makehardwarestation finalizes the plant before giving it back to us
    def callback(parser: Parser):
        plant = parser.plant()
        plant.set_contact_model(ContactModel.kHydroelasticWithFallback)
        scene_graph: SceneGraph = parser.scene_graph()
        config = scene_graph.get_config()
        if use_hydroelastic:
            # one object rigid, all else compliant? table
            # HE modulus parameters
            # allow broom to turn more easily

            # add "handle" to broom to make it easier to grip?
            config.default_proximity_properties.compliance_type = "compliant"
        scene_graph.set_config(config)

    station = MakeHardwareStation(scenario, meshcat=meshcat, parser_prefinalize_callback=callback)
    plant: MultibodyPlant = station.plant()


    builder = DiagramBuilder()
    builder.AddSystem(station)

    to_point_cloud = AddPointClouds(
        scenario=scenario, station=station, builder=builder, meshcat=meshcat
    )
    builder.ExportOutput(
        to_point_cloud["camera0"].point_cloud_output_port(), "point_cloud0"
    )

    # we add diagram after building
    # really scuffed but i don't care at this point
    meta_controller_sys = MetaController(plant, station)
    meta_controller = builder.AddSystem(meta_controller_sys)

    # ID CONTROLLER
    _, iiwa_plant, _ = build_temp_plant()
    ID_controller = builder.AddSystem(
        InverseDynamicsController(
            iiwa_plant,
            kp = np.ones(7) * 200,
            kd = np.ones(7) * 40,
            ki = np.zeros(7),
            has_reference_acceleration=False,
        )
    )

    control_estimated_state_input_port = ID_controller.get_input_port(0)
    control_desired_state_input_port = ID_controller.get_input_port(1)
    control_output_port = ID_controller.get_output_port(0)

    demux = builder.AddSystem(Demultiplexer([14, 1]))

    builder.Connect(
        meta_controller.get_output_port(),
        demux.get_input_port(),
    )

    builder.Connect(
        demux.get_output_port(0),
        control_desired_state_input_port
    )

    builder.Connect(
        demux.get_output_port(1), 
        station.GetInputPort("wsg.position")
    )

    builder.Connect(
        station.GetOutputPort("iiwa_state"),
        control_estimated_state_input_port
    )

    builder.Connect(
        control_output_port,
        station.GetInputPort("iiwa_actuation")
    )

    diagram = builder.Build()
    meta_controller_sys.add_diagram(diagram)

    return diagram, plant, station

# diagram = builder.Build()

In [ ]:
diagram, plant, station = make_diagram()

In [9]:

simulator = Simulator(diagram)
simulator.set_target_realtime_rate(1.0)
meshcat.StartRecording()
simulator.Initialize()
simulator.AdvanceTo(5.0)
meshcat.StopRecording()
meshcat.PublishRecording()

: 

In [ ]:
%tb